In [ ]:
#pip install tensorflow

In [ ]:
#pip install opencv-python

In [ ]:
# Importing necessary libraries
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

#dataset path setting 
dataset_path = os.listdir('C://Users//adity//OneDrive//Desktop//dataset')
print(dataset_path)
print ("types of classes labels found: ",len(dataset_path))

In [ ]:
# Creating a list to collect all data in single list 
class_labels =[]

for item in dataset_path:
    all_classes = os.listdir('C://Users//adity//OneDrive//Desktop//dataset' + '/' +item)
    #print(item,len(all_classes))
    for room in all_classes:
        class_labels.append((item, str('C://Users//adity//OneDrive//Desktop//dataset' + '/' +item) + '/' + room))
print (class_labels)

In [ ]:
# Creating dataframe 

df = pd.DataFrame(data=class_labels, columns = ['Labels','image'])
print (df.head())
print (df.tail())

In [ ]:
# Verifying data with labels
print ("total number of images in the datset: ", len(df))
label_count = df['Labels'].value_counts()
print (label_count)

In [ ]:
import cv2

In [ ]:
#resizing data since Efficient NetB0 requires 224dpi as resolution

path = 'C://Users//adity//OneDrive//Desktop//dataset//'
dataset_path = os.listdir('C://Users//adity//OneDrive//Desktop//dataset')

im_size = 224

images = []
labels = []

for i in dataset_path:
    data_path = path + str(i)
    filenames = [i for i in os.listdir(data_path)]
    for f in filenames:
        img = cv2.imread(data_path + '//' + f)
        new_img = cv2.resize(img,(im_size,im_size))
        images.append(new_img)
        labels.append(i)

In [ ]:
#craeting numpy array of images 
images = np.array(images)

images = images.astype('float32')/255.0        #array must be in 0 to 1 value
images.shape

In [ ]:
# label encoder: to encode labels in numbers
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
y = df['Labels'].values
print (y)

y_labelencoder = LabelEncoder()
y = y_labelencoder.fit_transform(y)
print (y)

In [ ]:
# OneHotEncoder: to create vectors of labels
y = y.reshape(-1,1)
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer([('my_ohe',OneHotEncoder(),[0])],remainder='passthrough')
Y= ct.fit_transform(y)
print(Y[:5])

In [ ]:
#splitting data into train data and test data

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

images,Y = shuffle (images, Y , random_state = 1)
train_x, test_x, train_y, test_y = train_test_split (images, Y, test_size = 0.05, random_state = 42)
print(train_x.shape)
print(test_x.shape)
print(train_y.shape)
print(test_y.shape)

Model Building

In [ ]:
#impoting EfficientNetB0
import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0

NUM_CLASSES = 3
IMG_SIZE = 224
size = (IMG_SIZE, IMG_SIZE)
inputs = layers.Input (shape= (IMG_SIZE,IMG_SIZE,3))

#using without transfer learning. Since, it is custom model. so, weights = None
outputs= EfficientNetB0(include_top = True, weights = None, classes= NUM_CLASSES)(inputs)

In [ ]:

model = tf.keras.Model (inputs,outputs)
opt = keras.optimizers.Adam(learning_rate =0.01)
model.compile(optimizer = opt , loss = "categorical_crossentropy", metrics = ["accuracy"])
model.summary()
hist = model.fit (train_x, train_y, epochs = 12 ,verbose = 2)

Plotting the Graph

In [ ]:
import matplotlib.pyplot as plt
def plot_hist(hist):
    plt.plot (hist.history["accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epochs")
    plt.legend(["train", "validation"], loc = "upper left")
    plt.show()

plot_hist(hist)

In [ ]:
preds = model.evaluate(test_x, test_y)
print ("Loss="+str(preds[0]))
print ("Test Accuracy = " + str (preds[1]))

Testing The Model

In [ ]:
from matplotlib.pyplot import imread, imshow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import decode_predictions, preprocess_input

img_path = "C:/Users/adity/OneDrive/Desktop/amla1.jpg"
img = cv2.imread(img_path)
img = cv2.resize(img, (224,224))

x = np.expand_dims(img,axis=0)
x = preprocess_input (x)

print ("input image shape: ", x.shape)

my_img = imread(img_path)
imshow (my_img)

In [ ]:
test = model.predict(x)
test